In [1]:
from pyspark.sql import SparkSession as s,functions as fcn
from pyspark import SparkContext

In [5]:
spark = (s
        .builder
        .appName("chapter3")
        .getOrCreate())

In [2]:
sc = SparkContext()

25/02/23 19:39:08 WARN Utils: Your hostname, Venugopals-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.220.234.156 instead (on interface en0)
25/02/23 19:39:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/23 19:39:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
dataRDD = (sc.parallelize(c=[("Brooke",20),("Danny",21),("Jules",20),("Charles",30),("Brooke",25)]))

In [4]:
agesRDD = (dataRDD.map(lambda x:(x[0],(x[1],1)))
            .reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1]))
            .map(lambda x: (x[0],x[1][0]/x[1][1],x[1][1]))
        )

agesRDD.collect()


/opt/anaconda3/envs/pyspark_practice/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py:154: DeprecationWarning: This process (pid=22306) is multi-threaded, use of fork() may lead to deadlocks in the child.


[('Brooke', 22.5, 2),
 ('Danny', 21.0, 1),
 ('Charles', 30.0, 1),
 ('Jules', 20.0, 1)]

25/02/23 19:39:20 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [9]:
df = spark.createDataFrame(dataRDD,["name","age"])

In [32]:
df.show()

+-------+---+
|   name|age|
+-------+---+
| Brooke| 20|
|  Danny| 21|
|  Jules| 20|
|Charles| 30|
| Brooke| 25|
+-------+---+



In [36]:
ages_df = (df
           .select("name","age")
           .groupBy("name")
           .agg(fcn.mean("age").alias("averageAge"),fcn.count("age").alias("countPeople"))
           )

In [37]:
ages_df.show(n=10)

+-------+----------+-----------+
|   name|averageAge|countPeople|
+-------+----------+-----------+
| Brooke|      22.5|          2|
|  Danny|      21.0|          1|
|  Jules|      20.0|          1|
|Charles|      30.0|          1|
+-------+----------+-----------+



In [47]:
df.createOrReplaceTempView("df")
ages_sql_query = (spark.sql('''SELECT 
                                    NAME, 
                                    MEAN(AGE),COUNT(*) AS AVERAGE_AGE 
                                FROM DF 
                                GROUP BY NAME''')
                  )

In [49]:
ages_sql_query.show()

+-------+---------+-----------+
|   NAME|mean(AGE)|AVERAGE_AGE|
+-------+---------+-----------+
| Brooke|     22.5|          2|
|  Danny|     21.0|          1|
|  Jules|     20.0|          1|
|Charles|     30.0|          1|
+-------+---------+-----------+

